In [30]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool,tool
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import (AgentExecutor,create_react_agent,ZeroShotAgent,create_tool_calling_agent)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory 
from langchain import hub

from llama_cpp import Llama
import sys
import os
sys.path.append('../../littleSeven/')
from common.config import cfg


from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

from langchain_experimental.chat_models import Llama2Chat
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

import json


from langchain_community.chat_models.llamacpp import ChatLlamaCpp

In [31]:
@tool
def get_current_time(*args, **kwargs):
    """Return the current time"""
    import datetime
    now = datetime.datetime.now()
    return json.dumps({
        "action": "Final Answer",  
        "action_input": f"The current time is {now.strftime('%I:%M %p')}"
    })

In [32]:
get_current_time.invoke({"input":"None"})

'{"action": "Final Answer", "action_input": "The current time is 03:19 PM"}'

In [33]:
tools=[get_current_time]

In [34]:
# def get_current_time(*args, **kwargs):
#     """Return the current time"""
#     import datetime
#     now = datetime.datetime.now()
#     return json.dumps({
#         "action": "Final Answer",  
#         "action_input": f"The current time is {now.strftime('%I:%M %p')}"
#     })

In [35]:
# tools = [
#     Tool(
#         name="Time",
#         func=get_current_time,
#         description="Useful if need the current time."
#     )
# ]

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [37]:
from os.path import expanduser

# from langchain_community.llms import LlamaCpp

model_path = expanduser(cfg.llm_gguf_model_path_2)
llm = ChatLlamaCpp(
    model_path=model_path,
    streaming=False,
)

llama_load_model_from_file: using device Metal (Apple M2 Max) - 21838 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model_gguf/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
l

In [38]:
# from os.path import expanduser

# from langchain_community.llms import LlamaCpp

# model_path = expanduser(cfg.llm_gguf_model_path_2)

# llm = LlamaCpp(
#     model_path=model_path,
#     streaming=False,
# )

In [39]:
llm.bind_tools(tools=tools)

RunnableBinding(bound=ChatLlamaCpp(client=<llama_cpp.llama.Llama object at 0x11e781010>, model_path='../model_gguf/llama-2-7b-chat.Q4_0.gguf', model_kwargs={}, streaming=False), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_current_time', 'description': 'Return the current time', 'parameters': {'properties': {'args': {'default': None, 'items': {}, 'type': 'array'}, 'kwargs': {'default': None, 'type': 'object'}}, 'type': 'object'}}}], 'tool_choice': None}, config={}, config_factories=[])

In [2]:
model = Llama2Chat(llm=llm)

In [3]:
# model.bind_tools(tools=tools)

In [4]:
# agent = create_tool_calling_agent(llm=model, tools=tools, prompt=prompt)